In [126]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, DataChunk

In [15]:
torch.set_printoptions(precision=3, linewidth=140, sci_mode=False)

In [16]:
batch_size = 1
num_seqs = 12
num_tokens = 1000

In [17]:
x = torch.randint(low=0, high=1000, size=(batch_size, num_seqs, num_tokens))
emb = nn.Embedding(10000, 8)
gru = nn.GRU(8, 16, batch_first=True, bidirectional=False, num_layers=1)
gru_2 = nn.GRU(16, 1, batch_first=True, bidirectional=False, num_layers=1)
lin = nn.Linear(1000, 1)

In [18]:
embedded = emb(x); print(x.shape)
h = torch.zeros(1, batch_size, 16); print(embedded.shape)
for i in range(num_seqs):
    embedded_one_seq = embedded[:, i]
    grued_one_seq, h = gru(embedded_one_seq, h)
print(grued_one_seq.shape)
squeezed_mini_batch, h_ = gru_2(grued_one_seq); print(squeezed_mini_batch.shape)
squeezed_mini_batch = squeezed_mini_batch.squeeze(-1); print(squeezed_mini_batch.shape)
result = lin(squeezed_mini_batch); print(result.shape)

torch.Size([1, 12, 1000])
torch.Size([1, 12, 1000, 8])
torch.Size([1, 1000, 16])
torch.Size([1, 1000, 1])
torch.Size([1, 1000])
torch.Size([1, 1])


In [19]:
l = [torch.randn(1000) for i in range(100)]
torch.stack(l).shape

torch.Size([100, 1000])

In [20]:
keke = torch.tensor([1, 2, 3])
keke_2 = keke.unsqueeze(-1)
keke.shape, keke_2.shape, keke.stride(), keke_2.stride(), keke_2

(torch.Size([3]),
 torch.Size([3, 1]),
 (1,),
 (1, 1),
 tensor([[1],
         [2],
         [3]]))

In [21]:
keke.storage(), keke_2.storage()

( 1
  2
  3
 [torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 3],
  1
  2
  3
 [torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 3])

In [22]:
loss = embedded_one_seq.mean()
loss.backward()

In [185]:
import random

class RandomSizeDataset(Dataset):
    def __init__(self, min = 1000, max=10000) -> None:
        super().__init__()
        self.min = min
        self.max = max

    def __getitem__(self, index):
        return torch.randint(1, 10000, (random.randint(self.min, self.max), )), random.randint(1, 10)

    def __len__(self):
        return 100

In [227]:
def collate_fn(batch):
    xs = [b[0] for b in batch]
    ys = [b[1] for b in batch]
    ls = [b[0].shape[0] for b in batch]
    padded = nn.utils.rnn.pad_sequence(xs, batch_first=True, padding_value=0)
    return padded, ls, torch.tensor(ys)

batch_size = 10

dl = DataLoader(
    dataset=RandomSizeDataset(1000, 100000),
    batch_size=batch_size,
    collate_fn=collate_fn
)

In [228]:
x, l, y = next(iter(dl))

In [232]:
x.shape

torch.Size([10, 97275])

In [238]:
emb = nn.Embedding(10000, 8).cuda()
gru = nn.GRU(8, 16, batch_first=True, bidirectional=False, num_layers=1).cuda()
gru_2 = nn.GRU(16, 1, batch_first=True, bidirectional=False, num_layers=1).cuda()
lin = nn.Linear(1000, 1).cuda()

In [239]:
def f(x, l):
    x = x.cuda()
    embedded = emb(x)
    packed = nn.utils.rnn.pack_padded_sequence(embedded, l, batch_first=True, enforce_sorted=False)
    h = torch.zeros(1, batch_size, 16).cuda()
    grued, h = gru(packed, h)
    grued_2, h = gru_2(grued)
    unpacked, unpacked_len = torch.nn.utils.rnn.pad_packed_sequence(grued_2)


In [240]:
%timeit f(x, l)

974 ms ± 4.49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
